# Vanuatu 5m DEM creation
Load in LiDAR where it exists and combine with FABDEM to produce 5m DEMs

In [6]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pathlib

import os
import sys
module_path = os.path.abspath(os.path.join("..", "src", "pacific-dems"))
if module_path not in sys.path:
    sys.path.append(module_path)

import create_dem_functions
import islands
import create_data_paths

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Define country and resolution

In [7]:
resolution = 5 # in meters
country_name = "vanuatu"

# Create paths and get contry information

In [8]:
paths = create_data_paths.get_paths(country_name=country_name, resolution=resolution)
islands_dict = islands.get_island(country_name)

# Specify any LiDAR data

In [10]:
# Path of LiDAR DEMs
efate_lidar_2012 = create_dem_functions.load_dem(paths["lidar"] / "dem_efate.tif")
espiritu_lidar_2012 = create_dem_functions.load_dem(paths["lidar"] / "dem_espiritu.tif")
efate_lidar_2021 = create_dem_functions.load_dem(paths["lidar"] / "EfateLiDAR2021-DEM-GRID-100_19358023_59S.tif")
malekula_lidar_2021 = create_dem_functions.load_dem(paths["lidar"] / "MalekulaLiDAR2021-HDEM-GRID-100_72708159_58S.tif")
tanna_lidar_2021 = create_dem_functions.load_dem(paths["lidar"] / "TannaLiDAR2021-DEM-GRID-100_31007823_59S.tif")

# Add to islands
islands_dict["shefa_province"]["lidar"] = [efate_lidar_2021, efate_lidar_2012]
islands_dict["sanma_province"]["lidar"] = espiritu_lidar_2012
islands_dict["malampa_province"]["lidar"] = malekula_lidar_2021
islands_dict["tafea_province"]["lidar"] = tanna_lidar_2021

# Get island outlines and combine FABDEMs

In [ ]:
for island_name, island_dict in islands_dict.items():
    print(f"Setup for {island_name}")
    island_dict["name"] = island_name
    island_dict["land"] = create_dem_functions.get_islands_in_boundary(island_dict=island_dict, gadm_path=paths["land"], output_path=paths["output"] / island_name)
    island_dict["fab"] = create_dem_functions.combine_fabs_in_boundary(island_dict=island_dict, fab_path=paths["fabdem"])

Setup for malampa_province
Setup for penama_province
Setup for tafea_province
/tWarning FABDEM tile S21E168_FABDEM_V1-2.tif doesn't exist in folder S30E160-S20E170_FABDEM_V1-2
/tWarning FABDEM tile S21E170_FABDEM_V1-2.tif doesn't exist in folder S30E170-S20W180_FABDEM_V1-2
/tWarning FABDEM tile S20E168_FABDEM_V1-2.tif doesn't exist in folder S20E160-S10E170_FABDEM_V1-2
/tWarning FABDEM tile S19E170_FABDEM_V1-2.tif doesn't exist in folder S20E170-S10W180_FABDEM_V1-2
Setup for torba_province
/tWarning FABDEM tile S14E168_FABDEM_V1-2.tif doesn't exist in folder S20E160-S10E170_FABDEM_V1-2
Setup for shefa_province


# Create DEMs over islands

In [ ]:
create_dem_functions.creating_dems_all_islands(island_groups=islands_dict, resolution=resolution, output_path=paths["output"])